In [12]:
import pandas as pd
import sys
sys.path.append('../src')

In [13]:
from data_processing import load_csv
output_df = load_csv('../data/processed/output2.csv')

In [14]:
output_df.columns

Index(['CustomerId', 'total_amount', 'customer_transactions',
       'fraud_transactions', 'total_fraud_amount', 'fraud_ratio',
       'fraud_amount_ratio', 'fraud_frequency_score', 'high_value_fraud',
       'last_transaction_date', 'Recency', 'Frequency', 'Monetary', 'cluster',
       'credit_risk'],
      dtype='object')

In [15]:
# drop CustomerId and credit_risk columns
X_train = output_df.drop(columns=['CustomerId', 'credit_risk'])

In [16]:
from sklearn.model_selection import train_test_split

# Drop unneeded columns
features = output_df.drop(columns=[
    'CustomerId', 'last_transaction_date', 'cluster', 'credit_risk',
    'fraud_transactions', 'total_fraud_amount', 'fraud_ratio',
    'fraud_amount_ratio', 'fraud_frequency_score', 'high_value_fraud'
])
X = features
y = output_df['credit_risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
features.columns

Index(['total_amount', 'customer_transactions', 'Recency', 'Frequency',
       'Monetary'],
      dtype='object')

In [18]:
X_train = X_train.dropna()
y_train = y_train.loc[X_train.index]  # make sure target matches
X_test = X_test.dropna()
y_test = y_test.loc[X_test.index]


In [19]:
print(y.value_counts())

credit_risk
0    2315
1    1427
Name: count, dtype: int64


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model_stat(models, X_train, y_train, X_test, y_test)

NameError: name 'models' is not defined

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(random_state=42)
}

model_stat(models, X_train, y_train, X_test, y_test)


Model: LogisticRegression
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC-AUC: 1.0
Model: RandomForest
Accuracy: 0.9986648865153538
Precision: 1.0
Recall: 0.9966216216216216
F1 Score: 0.9983079526226735
ROC-AUC: 1.0


In [10]:
print(X_test.columns)

Index(['total_amount', 'customer_transactions', 'fraud_transactions',
       'total_fraud_amount', 'fraud_ratio', 'fraud_amount_ratio',
       'fraud_frequency_score', 'high_value_fraud', 'Recency', 'Frequency',
       'Monetary'],
      dtype='object')


In [23]:
print(y.value_counts())
print(X.columns)


credit_risk
0    2315
1    1427
Name: count, dtype: int64
Index(['total_amount', 'customer_transactions', 'fraud_transactions',
       'total_fraud_amount', 'fraud_ratio', 'fraud_amount_ratio',
       'fraud_frequency_score', 'high_value_fraud', 'Recency', 'Frequency',
       'Monetary'],
      dtype='object')


In [7]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5, 10]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_


In [8]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("credit-risk-modeling")

with mlflow.start_run():
    mlflow.log_params(grid.best_params_)
    mlflow.log_metric("accuracy", accuracy_score(y_test, best_model.predict(X_test)))
    mlflow.sklearn.log_model(best_model, "model")
    mlflow.register_model("runs:/" + mlflow.active_run().info.run_id + "/model", "CreditRiskModel")


2025/07/02 20:11:32 INFO mlflow.tracking.fluent: Experiment with name 'credit-risk-modeling' does not exist. Creating a new experiment.
2025/07/02 20:11:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/02 20:11:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'CreditRiskModel'.
2025/07/02 20:11:37 WARNING mlflow.tracking._model_registry.fluent: Run with id 7abfcead2f8f495dbc43d5aec23bffee has no artifacts at artifact path 'model', registering model based on models:/m-b2da65f5d0164add9f4451e0ec371923 instead
Created version '1' of model 'CreditRiskModel'.
